### Imports und Datei

In [19]:
import osmium as osm
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import networkx as nx
import seaborn as sns
import numpy as np
import os
import requests
from collections import Counter
from math import radians, cos, sin, asin, sqrt
import xml.etree.ElementTree as ET
import math

# Setze den Pfad zum Dataset
dataset_path = os.path.join("Datasets", "unterfranken-latest.osm")
city_name = "Glattbach (village)"   

### Alle Daten aus dem Datensatz auslesen

In [ ]:
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []

    def tag_inventory(self, elem, elem_type, lon=None, lat=None, node_ids=None):
        if len(elem.tags) == 0:
            # Für Knoten ohne Tags
            self.osm_data.append([elem_type, 
                                  elem.id, 
                                  elem.visible,
                                  0, 
                                  None, 
                                  None, 
                                  lon, 
                                  lat, 
                                  node_ids])
        else:
            for tag in elem.tags:
                self.osm_data.append([elem_type, 
                                      elem.id,
                                      elem.visible,
                                      len(elem.tags),
                                      tag.k, 
                                      tag.v, lon, lat, node_ids])

    def node(self, n):
        self.tag_inventory(n, "node", lon=n.location.lon, lat=n.location.lat)

    def way(self, w):
        node_ids = [n.ref for n in w.nodes]  
        self.tag_inventory(w, "way", node_ids=node_ids)

# Daten auslesen
osmhandler = OSMHandler()
osmhandler.apply_file(dataset_path)

# DataFrame erstellen
data_colnames = ['type', 'id', 'visible', 'ntags', 'tagkey', 'tagvalue', 'longitude', 'latitude', 'node_ids']
df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
df_osm

### Datensatz nach Wegen gefiltert

In [ ]:
ways_filter = df_osm[(df_osm['type'] == 'way') & (df_osm['tagkey'] == 'highway')]
ways_filter

### API-Anfrage für die Längen- und Breitengrade der Städte

In [ ]:
# API-Anfrage um Bounding Box der Stadt zu bekommen
def get_bounding_box(city_name):
    url = f"https://nominatim.openstreetmap.org/search?city={city_name}&format=json"
    headers = {
        "User-Agent": "Bachelorproject/1.0"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        results = response.json()
        for result in results:
            if result.get("osm_type") == "relation" and 'boundingbox' in result:
                bbox = result['boundingbox']
                return [float(coord) for coord in bbox]  
    raise ValueError("Bounding Box konnte nicht abgerufen werden.")
    
bbox = get_bounding_box(city_name)
south, north, west, east = bbox

df_filtered_nodes = df_osm[
    ((df_osm['latitude'] >= south) & (df_osm['latitude'] <= north)) &
    ((df_osm['longitude'] >= west) & (df_osm['longitude'] <= east))    
]

df_filtered_nodes

### Daten in XML-Datei speichern

In [2]:
def save_to_xml(df, city_name):
    root = ET.Element("osm", version="0.6", generator="Bachelorproject")
    
    # Knoten hinzufügen
    for _, row in df[df['type'] == 'node'].iterrows():
        node = ET.SubElement(root, "node", id=str(row['id']),
                             visible=str(row['visible']),
                             lat=str(row['latitude']),
                             lon=str(row['longitude']))
        if row['tagkey']:
            ET.SubElement(node, "tag", k=row['tagkey'], v=row['tagvalue'])
    
    # Wege hinzufügen
    for _, row in df[df['type'] == 'way'].iterrows():
        way_attrs = {
            "id": str(row['id']),
            "visible": str(row['visible'])
        }
        # Wenn eine Länge vorhanden ist, als Attribut hinzufügen
        if 'length' in row and pd.notna(row['length']):
            way_attrs['length'] = str(row['length'])
        
        way = ET.SubElement(root, "way", **way_attrs)
        
        for node_id in row['node_ids']:
            ET.SubElement(way, "nd", ref=str(node_id))
        
        if row['tagkey']:
            ET.SubElement(way, "tag", k=row['tagkey'], v=row['tagvalue'])

    # XML-Baum erstellen und speichern
    city_folder = os.path.join("City_data")
    os.makedirs(city_folder, exist_ok=True)
    tree = ET.ElementTree(root)
    file_path = os.path.join(city_folder, f"{city_name}.osm")
    tree.write(file_path, encoding="utf-8", xml_declaration=True)
    print(f"Datei erfolgreich gespeichert: {file_path}")

### Daten aus der XML-Datei lesen

In [14]:
def load_from_xml(city_name):
    file_path = os.path.join("City_data", f"{city_name}.osm")
    tree = ET.parse(file_path)
    root = tree.getroot()

    osm_data = []

    # Knoten extrahieren
    for node in root.findall("node"):
        node_id = int(node.get("id"))
        lon = float(node.get("lon"))
        lat = float(node.get("lat"))
        osm_data.append(["node", node_id, True, 0, None, None, lon, lat, None])

    # Wege extrahieren
    for way in root.findall("way"):
        way_id = int(way.get("id"))
        node_ids = [int(nd.get("ref")) for nd in way.findall("nd")]
        for tag in way.findall("tag"):
            tagkey = tag.get("k")
            tagvalue = tag.get("v")
            osm_data.append(["way", way_id, True, len(way.findall("tag")), tagkey, tagvalue, None, None, node_ids])

    # DataFrame erstellen
    columns = ['type', 'id', 'visible', 'ntags', 'tagkey', 'tagvalue', 'longitude', 'latitude', 'node_ids']
    return pd.DataFrame(osm_data, columns=columns)

def load_from_xml_with_length(city_name):
    file_path = os.path.join("City_data", f"{city_name}.osm")
    tree = ET.parse(file_path)
    root = tree.getroot()

    osm_data = []

    # Knoten extrahieren
    for node in root.findall("node"):
        node_id = int(node.get("id"))
        lon = float(node.get("lon"))
        lat = float(node.get("lat"))
        osm_data.append(["node", node_id, True, 0, None, None, lon, lat, None])

    # Wege extrahieren
    for way in root.findall("way"):
        way_id = int(way.get("id"))
        node_ids = [int(nd.get("ref")) for nd in way.findall("nd")]
        length = way.get("length")  
        length = float(length) if length else None  
        for tag in way.findall("tag"):
            tagkey = tag.get("k")
            tagvalue = tag.get("v")
            osm_data.append(["way", way_id, True, len(way.findall("tag")), tagkey, tagvalue, None, None, node_ids, length])

    # DataFrame erstellen
    columns = ['type', 'id', 'visible', 'ntags', 'tagkey', 'tagvalue', 'longitude', 'latitude', 'node_ids', 'length']
    return pd.DataFrame(osm_data, columns=columns)

### Datensatz nach Städten filtern

In [ ]:
ways_filter = ways_filter[ways_filter['node_ids'].apply(lambda node_ids: any(node in df_filtered_nodes['id'].values for node in node_ids))]
df_filtered = pd.concat([df_filtered_nodes, ways_filter])
df_filtered.fillna('')
save_to_xml(df_filtered, city_name)
df_filtered

### Haversine-Formel

In [4]:
def haversine(lon1, lat1, lon2, lat2):
    # Dezimalzahlen in Radianten umwandeln
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine Formel
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius der Erde in Kilometer
    return c * r * 1000

In [ ]:
# Straßenlänge berechnen
def calculate_full_road_length(way_row, df_nodes):
    node_ids = way_row['node_ids']
    total_length = 0

    for i in range(len(node_ids) - 1):
        first_node = df_nodes[df_nodes['id'] == node_ids[i]]
        second_node = df_nodes[df_nodes['id'] == node_ids[i + 1]]

        # Überprüfung, ob beide Knoten existieren
        if first_node.empty or second_node.empty:
            return None

        total_length += haversine(
            first_node['longitude'].values[0], first_node['latitude'].values[0],
            second_node['longitude'].values[0], second_node['latitude'].values[0]
        )

    return total_length

df_city_data = load_from_xml(city_name)
valid_ways = df_city_data[df_city_data['type'] == 'way'].dropna(subset=['node_ids'])

road_lengths = []

for index, way_row in valid_ways.iterrows():
    length = calculate_full_road_length(way_row, df_city_data)
    if length is not None:
        road_lengths.append({'id': way_row['id'], 'length': length})

df_road_length = pd.DataFrame(road_lengths)
df_city_data = df_city_data.merge(df_road_length, left_on='id', right_on='id', how='left')
save_to_xml(df_city_data, city_name)

### Kreuzungsknoten erhalten

In [ ]:
df_city_data = load_from_xml_with_length(city_name)
all_node_ids = []
for node_ids_list in df_city_data[df_city_data['type'] == 'way']['node_ids']:
    if node_ids_list is not None:
        all_node_ids.extend(node_ids_list)

node_reference_count = Counter(all_node_ids)
nodes_referenced_multiple = {node_id for node_id, count in node_reference_count.items() if count > 1}

df_nodes_filtered_multiple = df_city_data[(df_city_data['type'] == 'node') & (df_city_data['id'].isin(nodes_referenced_multiple))]

filtered_data_multiple = pd.concat([df_city_data[df_city_data['type'] == 'way'], df_nodes_filtered_multiple], ignore_index=True)

save_to_xml(filtered_data_multiple, city_name)

### Graph vor dem Entfernen der Grad-2-Knoten

In [ ]:
df_city_data = load_from_xml(city_name)

G = nx.Graph()

# Knoten zum Graphen hinzufügen
for index, row in df_city_data[df_city_data['type'] == 'node'].iterrows():  
    G.add_node(row['id'], pos=(row['longitude'], row['latitude']))

# Kanten zum Graphen hinzufügen
for index, way in df_city_data[df_city_data['type'] == 'way'].iterrows():
    node_ids_list = way['node_ids']
    if node_ids_list is not None:
        filtered_node_ids_list = [node_id for node_id in node_ids_list if G.has_node(node_id)]
        if len(filtered_node_ids_list) > 1:
            for i in range(len(filtered_node_ids_list) - 1):
                is_dashed_highway = way['tagkey'] == 'highway' and way['tagvalue'] not in ['residential', 'unclassified', 'service', 'tertiary', 'secondary']
                G.add_edge(
                    filtered_node_ids_list[i], filtered_node_ids_list[i + 1],
                    street_name=way['tagvalue'],
                    is_dashed_highway=is_dashed_highway
                )

pos = nx.get_node_attributes(G, 'pos')

plt.figure(figsize=(12, 8))

# Normale Kanten zeichnen
edges_normal = [(u, v) for u, v, d in G.edges(data=True) if not d.get('is_dashed_highway')]
nx.draw_networkx_edges(G, pos, edgelist=edges_normal, edge_color='gray', width=1.5)

# Gestrichelte Kanten zeichnen
edges_dashed = [(u, v) for u, v, d in G.edges(data=True) if d.get('is_dashed_highway')]
nx.draw_networkx_edges(G, pos, edgelist=edges_dashed, edge_color='gray', width=1.5, style='--')

node_degrees = dict(G.degree())
node_color = ['red' if node_degrees[node] >= 3 else 'blue' for node in G.nodes()]
nx.draw_networkx_nodes(G, pos, node_size=10, node_color=node_color)

plt.title(city_name)
plt.xlabel("Längengrad")
plt.ylabel("Breitengrad")
plt.show()

### Knoten vom Grad 2 entfernen

#### Funktionen für die Bearbeitung der Grad 2 Knoten

In [84]:
G = nx.Graph()

neighbors_found = {}

# Entfernen von Knoten mit Grad 2 aus den node-ids
def remove_degree_2_nodes(node_ids):
    if node_ids is not None:
        return [node_id for node_id in node_ids if node_id not in degree_2_node_ids]
    return node_ids

# 2 Grad-3+-Nachbarn finden
def find_neighbors_with_degree_3(node_id, visited=None, found_neighbors=None):
    if visited is None:
        visited = set()
    if found_neighbors is None:
        found_neighbors = []

    visited.add(node_id)
    neighbors = list(G.neighbors(node_id))

    for neighbor in neighbors:
        if neighbor not in visited:
            neighbor_degree = G.degree(neighbor)

            # Füge Nachbarn hinzu, wenn er Grad 3 oder höher hat
            if neighbor_degree >= 3:
                found_neighbors.append(neighbor)

                if node_id not in neighbors_found:
                    neighbors_found[node_id] = []
                neighbors_found[node_id].append(neighbor)

                # Stoppe, wenn zwei passende Nachbarn gefunden wurden
                if len(found_neighbors) == 2:
                    return found_neighbors

            # Rekursive Suche fortsetzen, wenn der Nachbar Grad 2 hat
            elif neighbor_degree == 2:
                result = find_neighbors_with_degree_3(neighbor, visited, found_neighbors)
                if len(result) == 2:
                    return result
                
    return found_neighbors

# Aktualisieren der node_ids der Straßen, die Knoten mit Grad 2 enthalten
def update_node_ids(row, degree_2_node_ids):
    node_ids = row['node_ids']
    if node_ids is not None:
        updated_node_ids = list(node_ids)
        for node_id in node_ids:
            if node_id in degree_2_node_ids:
                # Finde Nachbarn des Knotens mit Grad 2
                neighbors = find_neighbors_with_degree_3(node_id)
                if len(neighbors) == 2:
                    neighbor_1, neighbor_2 = neighbors

                    # Finde die zwei Straßen, die den Grad-2-Knoten referenzieren
                    referencing_ways = ways_filter[ways_filter['node_ids'].apply(lambda x: node_id in x)]

                    if len(referencing_ways) == 2:
                        way_1, way_2 = referencing_ways.iloc[0], referencing_ways.iloc[1]

                        # Kopiere die Node-IDs als Listen
                        way_1_node_ids = list(way_1['node_ids']) if isinstance(way_1['node_ids'], list) else way_1['node_ids']
                        way_2_node_ids = list(way_2['node_ids']) if isinstance(way_2['node_ids'], list) else way_2['node_ids']

                        # Füge Nachbarn hinzu, falls sie nicht bereits vorhanden sind
                        if neighbor_1 not in way_2_node_ids:
                            way_2_node_ids.append(neighbor_1)
                        if neighbor_1 not in way_1_node_ids:
                            way_1_node_ids.append(neighbor_1)

                        # Aktualisiere die DataFrame-Zeilen
                        ways_filter.at[way_1.name, 'node_ids'] = way_1_node_ids
                        ways_filter.at[way_2.name, 'node_ids'] = way_2_node_ids

        # Stelle sicher, dass die Rückgabewerte korrekt sind
        return updated_node_ids if updated_node_ids else None
    return node_ids

#### Funktionen für die Bearbeitung der speziellen Grad-2-Knoten

In [85]:
def handle_special_degree_2_nodes(special_nodes, connected_ways, G):
    neighbors_by_special_node = {}

    def find_special_neighbors(node_id, connected_node_ids, visited=None, found_neighbors=None):
        if visited is None:
            visited = set()
        if found_neighbors is None:
            found_neighbors = set()

        visited.add(node_id)
        neighbors = list(G.neighbors(node_id))

        for neighbor in neighbors:
            if neighbor not in visited and neighbor in connected_node_ids:  
                neighbor_degree = G.degree(neighbor)

                # Füge Nachbarn hinzu, wenn er Grad 3 oder höher hat
                if neighbor_degree >= 3:
                    found_neighbors.add(neighbor) 

                    # Stoppe, wenn zwei passende Nachbarn gefunden wurden
                    if len(found_neighbors) == 2:
                        return list(found_neighbors)

                # Rekursive Suche fortsetzen, wenn der Nachbar Grad 2 hat
                elif neighbor_degree == 2:
                    result = find_special_neighbors(neighbor, connected_node_ids, visited, found_neighbors)
                    if len(result) == 2:
                        return list(result)

        return list(found_neighbors)

    # Für jeden speziellen Knoten die Nachbarn durch verbundene Wege finden
    for node_id, ways in connected_ways.items():
        neighbors_by_special_node[node_id] = []
        connected_node_ids = set()
        for way_id in ways:
            way = ways_filter[ways_filter['id'] == way_id]
            if not way.empty:
                connected_node_ids.update(way.iloc[0]['node_ids'])

        # Suche Nachbarn des speziellen Knotens, die Grad 3+ haben und in connected_node_ids liegen
        neighbors = find_special_neighbors(node_id, connected_node_ids)
        if neighbors:
            neighbors_by_special_node[node_id].extend(neighbors)

    return neighbors_by_special_node

# Identifizieren der speziellen Grad-2-Knoten
def identify_special_degree_2_nodes(node_way_mapping):
    special_nodes = {node_id: ways for node_id, ways in node_way_mapping.items() if len(ways) > 2}
    return special_nodes

# Ermitteln der Straßen, die mit speziellen Grad-2-Knoten verbunden sind
def get_connected_ways_for_special_nodes(df_special_nodes, valid_ways, G):
    connected_ways_by_node = {}

    for _, node_row in df_special_nodes.iterrows():
        node_id = node_row['id']
        connected_ways_by_node[node_id] = []
        
        # Finde Wege, die mit diesem speziellen Knoten verbunden sind
        for _, way_row in valid_ways.iterrows():
            way_id = way_row['id']
            node_ids_list = way_row['node_ids']
            
            if node_id in node_ids_list:
                # Prüfen, ob die Knoten des Wegs im Graphen existieren
                for i in range(len(node_ids_list) - 1):
                    u, v = node_ids_list[i], node_ids_list[i + 1]
                    if G.has_edge(u, v):
                        connected_ways_by_node[node_id].append(way_id)
                        break

    return connected_ways_by_node

# Aktualisieren der node_ids für spezielle Knoten
def update_special_node_ids(row, special_nodes, neighbors_by_special_node, connected_ways_for_special_nodes):
    node_ids = row['node_ids']

    if not isinstance(node_ids, list):
        node_ids = []
    
    if node_ids:
        updated_node_ids = list(node_ids)
        
        for node_id in node_ids:
            if node_id in special_nodes:
                neighbors = neighbors_by_special_node.get(node_id, [])
        
                if len(neighbors) == 2:
                    neighbor_1, neighbor_2 = neighbors
                    referencing_ways = connected_ways_for_special_nodes.get(node_id, [])
                    if len(referencing_ways) == 2:
                        way_1_id, way_2_id = referencing_ways[0], referencing_ways[1]
                        
                        way_1 = ways_filter[ways_filter['id'] == way_1_id].iloc[0]
                        way_2 = ways_filter[ways_filter['id'] == way_2_id].iloc[0]

                        way_1_node_ids = list(way_1['node_ids']) if isinstance(way_1['node_ids'], list) else way_1['node_ids']
                        way_2_node_ids = list(way_2['node_ids']) if isinstance(way_2['node_ids'], list) else way_2['node_ids']

                        if neighbor_1 not in way_2_node_ids:
                            way_2_node_ids.append(neighbor_1)
                        if neighbor_1 not in way_1_node_ids:
                            way_1_node_ids.append(neighbor_1)

                        ways_filter.at[way_1.name, 'node_ids'] = way_1_node_ids
                        ways_filter.at[way_2.name, 'node_ids'] = way_2_node_ids
        
        return updated_node_ids if updated_node_ids else None
    
    return node_ids

# Entfernen der speziellen und normalen Knoten
def remove_special_nodes_from_ids(node_ids, special_node_ids):
    if node_ids is not None:
        return [node_id for node_id in node_ids if node_id not in special_node_ids]
    return node_ids

#### Aufruf zum Entfernen der Grad-2-Knoten

In [ ]:
df_city_data = load_from_xml_with_length(city_name)

# Extrahieren der Knoten_Ids
existing_node_ids = set(df_city_data[df_city_data['type'] == 'node']['id'])

G = nx.Graph()
# Wege durchgehen und Kanten im Graphen hinzufügen
for _, row in df_city_data[df_city_data['type'] == 'way'].iterrows():
    node_ids = row['node_ids']
    if node_ids is not None:
        valid_node_ids = [node_id for node_id in node_ids if node_id in existing_node_ids]
        for i in range(len(valid_node_ids) - 1):
            G.add_edge(valid_node_ids[i], valid_node_ids[i + 1])

# Knoten mit Grad 2 finden
node_degrees = dict(G.degree())
nodes_with_degree_2 = [node_id for node_id, degree in node_degrees.items() if degree == 2]

# Knoten mit Grad 2 in filtered_data_multiple filtern
df_nodes_degree_2 = df_city_data[(df_city_data['type'] == 'node') & (df_city_data['id'].isin(nodes_with_degree_2))]
degree_2_node_ids = set(df_nodes_degree_2['id'])

# Spezielle Knoten mit Grad 2 finden
nodes_degree_2 = [node for node, degree in G.degree() if degree == 2]
node_way_mapping = {node_id: [] for node_id in nodes_degree_2}
for index, way in df_city_data[df_city_data['type'] == 'way'].iterrows():
    node_ids_list = way['node_ids']
    if node_ids_list is not None:
        for node_id in nodes_degree_2:
            if node_id in node_ids_list:
                node_way_mapping[node_id].append(way['id'])
special_nodes = identify_special_degree_2_nodes(node_way_mapping)

# Datensatz für spezielle Knoten
special_node_ids = set(special_nodes.keys()) 
df_special_nodes = df_city_data[df_city_data['id'].isin(special_node_ids)]

# Straßen mit speziellen Knoten ermitteln
connected_ways_for_special_nodes = get_connected_ways_for_special_nodes(df_special_nodes, df_city_data[df_city_data['type'] == 'way'], G)

# Nachbarn von speziellen Knoten ermitteln
neighbors_by_special_node = handle_special_degree_2_nodes(df_special_nodes, connected_ways_for_special_nodes, G)

# Aktualisieren die node_ids der speziellen Straßen
df_city_data['node_ids'] = df_city_data.apply(lambda row: update_special_node_ids(row, special_nodes, neighbors_by_special_node, connected_ways_for_special_nodes), axis=1)

# Aktualisiere die node_ids der Straßen
df_city_data['node_ids'] = df_city_data.apply(lambda row: update_node_ids(row, degree_2_node_ids), axis=1)

# Entfernen der Knoten mit Grad 2 aus den node_ids der Straßen
df_city_data['node_ids'] = df_city_data['node_ids'].apply(lambda node_ids: remove_special_nodes_from_ids(node_ids, degree_2_node_ids.union(special_node_ids)))

# Entferne Knoten mit Grad 2 aus dem Datensatz
df_city_data = df_city_data[~df_city_data['id'].isin(degree_2_node_ids.union(special_node_ids))]

# Entferne unverbundene Knoten aus dem Datensatz
connected_nodes = set(G.nodes)
df_city_data = df_city_data[(df_city_data['type'] != 'node') | (df_city_data['id'].isin(connected_nodes))]

save_to_xml(df_city_data, city_name)

### Graph nach dem Entfernen der 2-Grad-Knoten

In [ ]:
df_city_data = load_from_xml_with_length(city_name)

G = nx.Graph()

# Knoten zum Graphen hinzufügen
for index, row in df_city_data[df_city_data['type'] == 'node'].iterrows():  
    G.add_node(row['id'], pos=(row['longitude'], row['latitude']))

# Kanten zum Graphen hinzufügen
for index, way in df_city_data[df_city_data['type'] == 'way'].iterrows():
    node_ids_list = way['node_ids']
    if node_ids_list is not None:
        filtered_node_ids_list = [node_id for node_id in node_ids_list if G.has_node(node_id)]
        if len(filtered_node_ids_list) > 1:
            for i in range(len(filtered_node_ids_list) - 1):
                is_dashed_highway = way['tagkey'] == 'highway' and way['tagvalue'] not in ['residential', 'unclassified', 'service', 'tertiary', 'secondary']
                G.add_edge(
                    filtered_node_ids_list[i], filtered_node_ids_list[i + 1],
                    street_name=way['tagvalue'],
                    is_dashed_highway=is_dashed_highway
                )

pos = nx.get_node_attributes(G, 'pos')

plt.figure(figsize=(12, 8))

# Normale Kanten zeichnen
edges_normal = [(u, v) for u, v, d in G.edges(data=True) if not d.get('is_dashed_highway')]
nx.draw_networkx_edges(G, pos, edgelist=edges_normal, edge_color='gray', width=1.5)

# Gestrichelte Kanten zeichnen
edges_dashed = [(u, v) for u, v, d in G.edges(data=True) if d.get('is_dashed_highway')]
nx.draw_networkx_edges(G, pos, edgelist=edges_dashed, edge_color='gray', width=1.5, style='--')

node_degrees = dict(G.degree())
node_color = ['red' if node_degrees[node] >= 3 else 'blue' for node in G.nodes()]
nx.draw_networkx_nodes(G, pos, node_size=10, node_color=node_color)

plt.title(city_name)
plt.xlabel("Längengrad")
plt.ylabel("Breitengrad")
plt.show()

### Knoten zusammenfügen, die nah beieinander stehen

In [ ]:
# Löschen der Knoten, die durch anderen ersetzt wurde
def remove_replaced_nodes(df_nodes, node_merge_dict):
    replaced_node_ids = set(node_merge_dict.keys()) - set(node_merge_dict.values())
    return df_nodes[~df_nodes['id'].isin(replaced_node_ids)]

# Aktualisiere node_ids
def update_node_ids_merged(node_ids, node_merge_dict):
    if node_ids is not None and isinstance(node_ids, list):
        updated_node_ids = [node_merge_dict.get(node_id, node_id) for node_id in node_ids]
        # Entferne doppelte Knoten, falls sie durch das Zusammenführen entstanden sind
        unique_node_ids = list(dict.fromkeys(updated_node_ids))
        return unique_node_ids
    return node_ids

# Zusammenfügen der Knoten, die nah beieinander stehen
def merge_within_distance(df_nodes, max_distance=5):
    merged_nodes = []
    node_merge_dict = {}
    seen = set()

    for i, node_1 in df_nodes.iterrows():
        for j, node_2 in df_nodes.iterrows():
            # Vergleich gleicher Knoten verhindern
            if node_1['id'] == node_2['id'] or node_1['id'] in seen or node_2['id'] in seen:
                continue

            lon1, lat1 = node_1['longitude'], node_1['latitude']
            lon2, lat2 = node_2['longitude'], node_2['latitude']

            distance = haversine(lon1, lat1, lon2, lat2)

            if distance < max_distance:
                merged_node = {
                    'id': node_1['id'],
                    'longitude': (lon1 + lon2) / 2,
                    'latitude': (lat1 + lat2) / 2
                }
                merged_nodes.append(merged_node)

                # Knoten als gesehen markiert
                seen.add(node_1['id'])
                seen.add(node_2['id'])

                node_merge_dict[node_2['id']] = node_1['id']

    return merged_nodes, node_merge_dict

df_city_data = load_from_xml_with_length(city_name)

df_nodes = df_city_data[df_city_data['type'] == 'node']
df_ways = df_city_data[df_city_data['type'] == 'way']

merged_nodes, node_merge_dict = merge_within_distance(df_nodes)
df_merged_nodes = pd.DataFrame(merged_nodes, columns=['id', 'longitude', 'latitude'])

# Wege aktualisieren
df_ways['node_ids'] = df_ways['node_ids'].apply(
    lambda x: update_node_ids_merged(x, node_merge_dict)
)
# Ehemalige Knoten entfernen
df_nodes_updated = remove_replaced_nodes(df_nodes, node_merge_dict)

df_nodes_final = pd.concat([df_nodes_updated, df_merged_nodes], ignore_index=True)

# Doppelte und leere Knoten-IDs entfernen
df_ways = df_ways[df_ways['node_ids'].apply(lambda x: len(x) > 1)]

df_city_data_updated = pd.concat([df_nodes_final, df_ways], ignore_index=True)

save_to_xml(df_city_data_updated, city_name)

### Graph nach Zusammenfügen der Knoten die beieinander stehen

In [ ]:
df_city_data = load_from_xml_with_length(city_name)

G = nx.Graph()

# Knoten zum Graphen hinzufügen
for index, row in df_city_data[df_city_data['type'] == 'node'].iterrows():  
    G.add_node(row['id'], pos=(row['longitude'], row['latitude']))

# Kanten zum Graphen hinzufügen
for index, way in df_city_data[df_city_data['type'] == 'way'].iterrows():
    node_ids_list = way['node_ids']
    if node_ids_list is not None:
        filtered_node_ids_list = [node_id for node_id in node_ids_list if G.has_node(node_id)]
        if len(filtered_node_ids_list) > 1:
            for i in range(len(filtered_node_ids_list) - 1):
                is_dashed_highway = way['tagkey'] == 'highway' and way['tagvalue'] not in ['residential', 'unclassified', 'service', 'tertiary', 'secondary']
                G.add_edge(
                    filtered_node_ids_list[i], filtered_node_ids_list[i + 1],
                    street_name=way['tagvalue'],
                    is_dashed_highway=is_dashed_highway
                )

pos = nx.get_node_attributes(G, 'pos')

plt.figure(figsize=(12, 8))

# Normale Kanten zeichnen
edges_normal = [(u, v) for u, v, d in G.edges(data=True) if not d.get('is_dashed_highway')]
nx.draw_networkx_edges(G, pos, edgelist=edges_normal, edge_color='gray', width=1.5)

# Gestrichelte Kanten zeichnen
edges_dashed = [(u, v) for u, v, d in G.edges(data=True) if d.get('is_dashed_highway')]
nx.draw_networkx_edges(G, pos, edgelist=edges_dashed, edge_color='gray', width=1.5, style='--')

node_degrees = dict(G.degree())
node_color = ['red' if node_degrees[node] >= 3 else 'blue' for node in G.nodes()]
nx.draw_networkx_nodes(G, pos, node_size=10, node_color=node_color)

plt.title(city_name)
plt.xlabel("Längengrad")
plt.ylabel("Breitengrad")
plt.show()

In [ ]:
city_data = load_from_xml_with_length(city_name)
node_degrees = dict(G.degree()) 

node_x = []
node_y = []
node_text = []  
node_color = [] 

for index, row in city_data.iterrows():
    if pd.notnull(row['longitude']) and pd.notnull(row['latitude']):
        node_x.append(row['longitude'])
        node_y.append(row['latitude'])
        
        hover_info = f"ID: {row['id']}<br>Tag Key: {row['tagkey']}<br>Tag Value: {row['tagvalue']}<br>Degree: {node_degrees[row['id']]}"
        node_text.append(hover_info)
        
        node_color.append(node_degrees[row['id']])

edge_x = []
edge_y = []
dashed_edge_x = []
dashed_edge_y = []
edge_street_names = []

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]

    # Überprüfen, ob der Rand highway != 'residual' ist
    is_dashed_highway = False
    street_name = ""
    for street in city_data[city_data['type'] == 'way'].itertuples():
        if edge[0] in street.node_ids and edge[1] in street.node_ids:
            street_name = street.tagvalue
            # Highway-Kanten, die nicht 'residual' sind, werden gestrichelt
            if street.tagkey == "highway" and street.tagvalue not in ['residential', 'unclassified', 'service', 'tertiary', 'secondary']:
                is_dashed_highway = True
            break

    if is_dashed_highway:
        dashed_edge_x.extend([x0, x1, None])
        dashed_edge_y.extend([y0, y1, None])
    else:
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    # Berechnung der Mittelpunkte für die Straßennamen
    midpoint_x = (x0 + x1) / 2
    midpoint_y = (y0 + y1) / 2
    edge_street_names.append((midpoint_x, midpoint_y, street_name))


# Plot für durchgezogene Kanten
solid_edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Plot für gestrichelte Kanten (highways)
dashed_edge_trace = go.Scatter(
    x=dashed_edge_x, y=dashed_edge_y,
    line=dict(width=0.5, color='#888', dash='dash'),
    hoverinfo='none',
    mode='lines'
)

# Plot für Knoten
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        color=node_color,
        size=10,
        colorbar=dict(
            thickness=15,
            title='Knoten Verbindungen',
            xanchor='left',
            titleside='right'
        )),
    text=node_text
)

# Hinzufügen der Straßennamen zwischen den Knoten
street_name_trace = go.Scatter(
    x=[name[0] for name in edge_street_names],
    y=[name[1] for name in edge_street_names],
    text=[name[2] for name in edge_street_names],
    mode='text',
    textfont=dict(
        size=12,
        color='darkblue'
    ),
    hoverinfo='text'
)

layout = go.Layout(
    title=city_name,
    titlefont=dict(size=16),
    showlegend=False,
    hovermode='closest',
    margin=dict(b=0, l=0, r=0, t=0),
    annotations=[dict(
        text=f"{city_name}",
        showarrow=False,
        xref="paper", yref="paper",
        x=0.005, y=-0.002)],
    xaxis=dict(showgrid=False, zeroline=False, visible=False),
    yaxis=dict(showgrid=False, zeroline=False, visible=False)
)

fig = go.Figure(data=[solid_edge_trace, dashed_edge_trace, node_trace], layout=layout)

fig.show()

# Auswertungen

### Boxplott der Straßenlängen

In [ ]:
def custom_sort(file):
    if "village" in file:
        return 0
    elif "town" in file:
        return 1
    elif "county" in file:
        return 2
    return 3

city_folder = "City_data"
files = [file for file in os.listdir(city_folder) if file.endswith(".osm")]

files.sort(key=custom_sort)

# Anzahl der Diagramme pro Zeile
cols = 3
rows = math.ceil(len(files) / cols)  # Anzahl der Zeilen berechnen
fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows), sharey=True)

# Achsen flachstellen für einfacheren Zugriff
axes = axes.flatten()

# Boxplots erstellen
for i, file in enumerate(files):
    city_name = os.path.splitext(file)[0]
    df = load_from_xml_with_length(city_name)

    # Nur Straßen mit Längen auswählen
    df_filtered = df[(df['type'] == 'way') & df['length'].notna()]

    sns.boxplot(data=df_filtered, y='length', ax=axes[i])
    axes[i].set_title(f"{city_name}")
    axes[i].set_xlabel("Straßen")
    axes[i].set_ylabel("Länge (m)")

# Nicht genutzte Subplots ausblenden
for i in range(len(files), len(axes)):
    fig.delaxes(axes[i])

# Gesamttitel und Darstellung
fig.suptitle("Straßenlängen pro Stadt", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)  # Platz für den Titel
plt.show()

### Histogramm zu den Straßenlängen

In [ ]:
def custom_sort(file):
    if "village" in file:
        return 0
    elif "town" in file:
        return 1
    elif "county" in file:
        return 2
    return 3

city_folder = "City_data"
files = [file for file in os.listdir(city_folder) if file.endswith(".osm")]

files.sort(key=custom_sort)

# Anzahl der Spalten und Zeilen berechnen
columns = 3
rows = math.ceil(len(files) / columns)

# Subplots erstellen
fig, axes = plt.subplots(rows, columns, figsize=(15, 5 * rows))

# Achsen flach machen, damit sie iteriert werden können
axes = axes.flatten()

for i, file in enumerate(files):
    city_name = os.path.splitext(file)[0]
    df = load_from_xml_with_length(city_name)

    # Daten filtern: Nur Straßen mit definierter Länge
    df_filtered = df[(df['type'] == 'way') & df['length'].notna()]

    sns.histplot(df_filtered['length'], bins=30, kde=True, ax=axes[i])
    axes[i].set_title(f"Straßenlängen in {city_name}")
    axes[i].set_xlabel("Straßenlänge (m)")
    axes[i].set_ylabel("Anzahl der Straßen")

# Leere Achsen entfernen, falls die Anzahl der Dateien nicht durch 3 teilbar ist
for j in range(len(files), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

### Histogramm zu den Bundesstraßen

In [ ]:
def custom_sort(file):
    if "village" in file:
        return 0
    elif "town" in file:
        return 1
    elif "county" in file:
        return 2
    return 3

city_folder = "City_data"
files = [file for file in os.listdir(city_folder) if file.endswith(".osm")]

files.sort(key=custom_sort)

# Farben für jeden Typ
highway_types = ['primary', 'secondary', 'tertiary']
colors = {'primary': 'skyblue', 'secondary': 'orange', 'tertiary': 'green'}

# Raster mit 3 Diagrammen pro Zeile
columns = 3
rows = math.ceil(len(files) / columns)

fig, axes = plt.subplots(rows, columns, figsize=(15, 5 * rows))
axes = axes.flatten()  # Alle Achsen in einer Liste flach machen

# Histogramme zeichnen
for ax, file in zip(axes, files):
    city_name = os.path.splitext(file)[0]
    df = load_from_xml_with_length(city_name)

    # Pro Highway-Typ zeichnen
    for highway in highway_types:
        df_filtered = df[(df['type'] == 'way') & 
                         (df['tagkey'] == 'highway') & 
                         (df['tagvalue'] == highway) & 
                         df['length'].notna()]

        if not df_filtered.empty:
            sns.histplot(
                df_filtered['length'], bins=30, kde=True, 
                ax=ax, color=colors[highway], label=highway
            )

    ax.set_title(f"Bundesstraßenlängen in {city_name}")
    ax.set_xlabel("Straßenlänge (m)")
    ax.set_ylabel("Anzahl der Straßen")
    ax.legend(title="Highway-Typ")

# Verbleibende Achsen ausblenden
for ax in axes[len(files):]:
    ax.axis('off')

plt.tight_layout()
plt.show()

### Histogramm zu Stadtstraßen

In [ ]:
def custom_sort(file):
    if "village" in file:
        return 0
    elif "town" in file:
        return 1
    elif "county" in file:
        return 2
    return 3

city_folder = "City_data"
files = [file for file in os.listdir(city_folder) if file.endswith(".osm")]

files.sort(key=custom_sort)

# Farben für jeden Typ
highway_types = ['residential', 'living_street']
colors = {'residential': 'skyblue', 'living_street': 'orange'}

# Rastergröße berechnen (3 Diagramme pro Zeile)
columns = 3
rows = math.ceil(len(files) / columns)

# Subplots erstellen
fig, axes = plt.subplots(rows, columns, figsize=(15, 5 * rows))
axes = axes.flatten()  # Zum einfachen Iterieren in einer 1D-Liste

# Histogramme zeichnen
for i, file in enumerate(files):
    city_name = os.path.splitext(file)[0]
    df = load_from_xml_with_length(city_name)

    ax = axes[i]  # Aktuellen Achsen auswählen

    # Pro Highway-Typ zeichnen
    for highway in highway_types:
        df_filtered = df[(df['type'] == 'way') & 
                         (df['tagkey'] == 'highway') & 
                         (df['tagvalue'] == highway) & 
                         df['length'].notna()]

        if not df_filtered.empty:
            sns.histplot(
                df_filtered['length'], bins=30, kde=True, 
                ax=ax, color=colors[highway], label=highway
            )

    ax.set_title(f"Stadtstraßenlängen in {city_name}")
    ax.set_xlabel("Straßenlänge (m)")
    ax.set_ylabel("Anzahl der Straßen")
    ax.legend(title="Highway-Typ")

# Unbenutzte Achsen ausblenden
for j in range(len(files), len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

### Histogramm für Fußwege

In [ ]:
def custom_sort(file):
    if "village" in file:
        return 0
    elif "town" in file:
        return 1
    elif "county" in file:
        return 2
    return 3

city_folder = "City_data"
files = [file for file in os.listdir(city_folder) if file.endswith(".osm")]

files.sort(key=custom_sort)

# Farben für jeden Typ
highway_types = ['service', 'pedestrian', 'track', 'footway', 'sidewalk']
colors = {'service': 'skyblue', 'pedestrian': 'orange', 'track': 'green', 'footway': 'yellow', 'sidewalk': 'red'}

# Anzahl der Zeilen und Spalten berechnen
cols = 3  # Drei Diagramme pro Zeile
rows = math.ceil(len(files) / cols)  # Berechnet die benötigten Zeilen

# Subplots erstellen
fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
axes = axes.flatten()  # Flach machen, um sie einfacher anzusprechen

# Histogramme zeichnen
for ax, file in zip(axes, files):
    city_name = os.path.splitext(file)[0]
    df = load_from_xml_with_length(city_name)

    # Pro Highway-Typ zeichnen
    for highway in highway_types:
        df_filtered = df[(df['type'] == 'way') & 
                         (df['tagkey'] == 'highway') & 
                         (df['tagvalue'] == highway) & 
                         df['length'].notna()]

        if not df_filtered.empty:
            sns.histplot(
                df_filtered['length'], bins=30, kde=True, 
                ax=ax, color=colors[highway], label=highway
            )

    ax.set_title(f"Längen von Fußgängerwegen in {city_name}")
    ax.set_xlabel("Straßenlänge (m)")
    ax.set_ylabel("Anzahl der Straßen")
    ax.legend(title="Highway-Typ")

# Entferne leere Subplots (falls weniger Dateien als Subplots vorhanden sind)
for ax in axes[len(files):]:
    ax.remove()

plt.tight_layout()
plt.show()

### Histogramm für Knotengrade

In [ ]:
def custom_sort(file):
    if "village" in file:
        return 0
    elif "town" in file:
        return 1
    elif "county" in file:
        return 2
    return 3

city_folder = "City_data"
files = [file for file in os.listdir(city_folder) if file.endswith(".osm")]

# Dateien sortieren
files.sort(key=custom_sort)

# Anzahl der Spalten für Subplots
columns = 3
rows = (len(files) + columns - 1) // columns  # Zeilenanzahl berechnen
fig, axes = plt.subplots(rows, columns, figsize=(15, 5 * rows), squeeze=False)

# Subplots erstellen
for idx, file in enumerate(files):
    city_name = os.path.splitext(file)[0]
    city_data = load_from_xml_with_length(city_name)  # Daten der aktuellen Datei laden

    # Netzwerkgraph für diese Stadt erstellen
    G = nx.Graph()
    for _, row in city_data[city_data['type'] == 'way'].iterrows():
        node_ids = row['node_ids']
        if node_ids is not None:
            for i in range(len(node_ids) - 1):
                G.add_edge(node_ids[i], node_ids[i + 1])

    # Knotengrade berechnen
    node_degrees = dict(G.degree())
    city_data['degree'] = city_data['id'].map(node_degrees).fillna(0).astype(int)
    city_data = city_data[city_data['degree'] > 0]  # Knoten ohne Verbindungen entfernen

    # Knotengrade auf maximal 6 beschränken
    city_data = city_data[city_data['degree'] <= 6]
    city_node_degrees = city_data['degree']

    # Position im Gitterlayout berechnen
    row, col = divmod(idx, columns)
    ax = axes[row][col]

    bins = np.arange(1, 7) - 0.5
    
    sns.histplot(city_node_degrees, bins=bins, kde=True, ax=ax)  # Anzahl der Bins anpassen
    ax.set_title(f"Knotengrade in {city_name}")
    ax.set_xlabel("Knotengrad")
    ax.set_ylabel("Anzahl der Knoten")
    ax.set_xticks(range(1, 7))

# Leere Plots ausblenden
for idx in range(len(files), rows * columns):
    row, col = divmod(idx, columns)
    axes[row][col].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
def calculate_grid_density(df, grid_size=100):
    earth_radius = 6371000
    df['x'] = np.radians(df['longitude']) * np.cos(np.radians(df['latitude'])) * earth_radius
    df['y'] = np.radians(df['latitude']) * earth_radius

    df['grid_x'] = (df['x'] // grid_size).astype(int)
    df['grid_y'] = (df['y'] // grid_size).astype(int)

    grid_density = df.groupby(['grid_x', 'grid_y']).size().reset_index(name='node_count')
    return grid_density    

def custom_sort(file):
    if "village" in file:
        return 0
    elif "town" in file:
        return 1
    elif "county" in file:
        return 2
    return 3

city_folder = "City_data"
files = [file for file in os.listdir(city_folder) if file.endswith(".osm")]
all_densities = []

for file in files:
    city_name = os.path.splitext(file)[0]
    df = load_from_xml_with_length(city_name)
    
    df_nodes = df[df['type'] == 'node']

    G = nx.Graph()
    for _, row in df[df['type'] == 'way'].iterrows():
        node_ids = row['node_ids']
        if node_ids is not None:
            for i in range(len(node_ids) - 1):
                G.add_edge(node_ids[i], node_ids[i + 1])
    
    node_degrees = dict(G.degree())
    df_nodes['degree'] = df_nodes['id'].map(node_degrees).fillna(0).astype(int)
    df_nodes = df_nodes[df_nodes['degree'] > 0]
    
    # Berechne Knotendichte
    grid_density = calculate_grid_density(df_nodes)
    grid_density['city_name'] = city_name
    all_densities.append(grid_density)

# Kombiniere die Knotendichten aller Städte
df_all_densities = pd.concat(all_densities, ignore_index=True)

plt.figure(figsize=(10, 6))
sns.boxplot(data=df_all_densities, x='city_name', y='node_count')
plt.title("Vergleich der Knotendichten im 100m-Gitternetz")
plt.xlabel("Stadt")
plt.ylabel("Knotendichte (Anzahl Knoten pro Gitterzelle)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()